In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Part 2
</center>
</h1>
<div class=h1_cell>
<p>
In this part we will explore the best model for predicting on our binary column. The two models I am going to explore are the random forest model using out of bag testing and the decision tree model using k-fold testing. Some of the exploring we will do involves changing parameters like number of trees for random forests and depth for decision trees to find the right balance and the best overall results.
</div>

In [38]:
import pandas as pd
import os

home_path = os.path.expanduser('~')

file_path = '/Documents/CIS 399/'

file_name = 'wrangled_game_table.csv'

game_table = pd.read_csv(home_path + file_path + file_name)
pd.__version__

u'0.20.3'

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
os.chdir(home_path + '/Documents/CIS 399/funcs/datascience_1')
!git pull

Already up to date.


In [6]:
import sys
sys.path.append(home_path + '/Documents/CIS 399/funcs/datascience_1')
from week7 import *
%who function

accuracy	 build_pred	 build_tree_iter	 caser	 closest_centroid	 compute_mean	 compute_prediction	 compute_training	 euclidean_distance	 
f1	 find_best_splitter	 forest_builder	 generate_table	 gig	 gini	 informedness	 initialize_centroids	 k_fold	 
k_means	 phase_1	 phase_2	 predictor_case	 probabilities	 row_to_vect	 seeder	 tree_predictor	 vote_taker	 



In [39]:
game_table.head(-1)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70,nas_binned,nas_low,nas_average,nas_high,nas_nan,eus_binned,eus_low,eus_average,eus_high,eus_nan,jps_binned,jps_low,jps_average,jps_high,jps_nan,ots_binned,ots_low,ots_average,ots_high,ots_nan,gls_binned,gls_low,gls_average,gls_high,gls_nan
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,2006.000000,76.000000,1,high,0,0,1,0,high,0,0,1,0,average,0,1,0,0,high,0,0,1,0,high,0,0,1,0
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.550000,1,high,0,0,1,0,low,1,0,0,0,average,0,1,0,0,low,1,0,0,0,average,0,1,0,0
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E,2008.000000,82.000000,1,average,0,1,0,0,average,0,1,0,0,average,0,1,0,0,low,1,0,0,0,average,0,1,0,0
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E,2009.000000,80.000000,1,average,0,1,0,0,average,0,1,0,0,low,1,0,0,0,low,1,0,0,0,average,0,1,0,0
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.550000,1,low,1,0,0,0,low,1,0,0,0,high,0,0,1,0,low,1,0,0,0,average,0,1,0,0
5,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.550000,1,average,0,1,0,0,low,1,0,0,0,average,0,1,0,0,low,1,0,0,0,average,0,1,0,0
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E,2006.000000,89.000000,1,low,1,0,0,0,low,1,0,0,0,average,0,1,0,0,low,1,0,0,0,average,0,1,0,0
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E,2006.000000,58.000000,0,average,0,1,0,0,low,1,0,0,0,low,1,0,0,0,low,1,0,0,0,average,0,1,0,0
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.44,6.94,4.70,2.24,28.32,87.0,80.0,8.4,594.0,Nintendo,E,2009.000000,87.000000,1,average,0,1,0,0,low,1,0,0,0,average,0,1,0,0,low,1,0,0,0,average,0,1,0,0
9,Duck Hunt,NES,1984.0,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.550000,1,average,0,1,0,0,low,1,0,0,0,low,1,0,0,0,low,1,0,0,0,average,0,1,0,0


<h1>

Random Forest Testing

</h1>
<div class=h1_cell>
<p>
I am going to explore random forest with different parameters. More specifically, we will be exploring the effects of adding more trees to the forest. 
<p>
<p>
These are the columns to use for our forest_builder function.
</div>

In [40]:
forest_columns = [
    #North American Sales
    'nas_low', 'nas_average', 'nas_high',
    #European Sales
    'eus_low', 'eus_average', 'eus_high',
    #Japanese Sales
    'jps_low', 'jps_average', 'jps_high',
    #Other Sales
    'ots_low', 'ots_average', 'ots_high',
    #Global Sales
    'gls_low', 'gls_average', 'gls_high'
]

In [41]:
#Default tree number is 5
forest1 = forest_builder(game_table, forest_columns, 'rating_over_70', hypers={})
len(forest1)

5

<hr>
<h2>
Out of Bag Testing
</h2>
<p>
<div class=h1_cell>
<p>
We're going to use out of bag testing to test the results of our forest.

In [42]:
# I wrote this function for oob testing purposes
def oob_testing(forest, table):
    new_table = table.loc[forest[0]['oob']]
    
    for i in range(1, len(forest)):
        temp = table.loc[forest[i]['oob']]
        new_table = new_table.append(temp)
    return new_table

In [44]:
oob_table1 = oob_testing(forest1, game_table)
forest1_oob = forest_builder(oob_table1, forest_columns, 'rating_over_70', hypers={})

game_table['forest1_oob'] = game_table.apply(lambda row: vote_taker(row, forest1_oob), axis=1)
game_table['forest1_oob_types'] = game_table.apply(lambda row: predictor_case(row, pred='forest1_oob', target='rating_over_70'), axis=1)
forest1_oob_types = game_table['forest1_oob_types'].value_counts()
print(forest1_oob_types.sum())  # length of testing table
print((accuracy(forest1_oob_types), f1(forest1_oob_types), informedness(forest1_oob_types)))

16719
(0.75267659549016086, 0.0048134777376654635, 0.0024125452352230514)


<hr>
<h2>
Experiment with different sizes</h2>
<p>
<div class=h1_cell>
<p>
We're going to continue using oob testing with different sizes.

In [45]:
#10 trees
forest2 = forest_builder(game_table, forest_columns, 'rating_over_70', hypers={'total-trees':10})
len(forest2)

10

In [46]:
# OOB testing
oob_table2 = oob_testing(forest2, game_table)
forest2_oob = forest_builder(oob_table2, forest_columns, 'rating_over_70', hypers={'total-trees':10})

game_table['forest2_oob'] = game_table.apply(lambda row: vote_taker(row, forest2_oob), axis=1)
game_table['forest2_oob_types'] = game_table.apply(lambda row: predictor_case(row, pred='forest2_oob', target='rating_over_70'), axis=1)
forest2_oob_types = game_table['forest2_oob_types'].value_counts()
print(forest2_oob_types.sum())  # length of testing table
print((accuracy(forest2_oob_types), f1(forest2_oob_types), informedness(forest2_oob_types)))

16719
(0.75327471738740359, 0.012921751615218951, 0.005957167824620635)


<div class=h1_cell>
<p>
All measures, accuracy, F1 score, and informedness, increased with more trees.
<p>
We'll continue adding trees.
</div>

In [47]:
#15 trees
forest3 = forest_builder(game_table, forest_columns, 'rating_over_70', hypers={'total-trees':15})
len(forest3)

15

In [48]:
# OOB testing
oob_table3 = oob_testing(forest3, game_table)
forest3_oob = forest_builder(oob_table3, forest_columns, 'rating_over_70', hypers={'total-trees':15})

game_table['forest3_oob'] = game_table.apply(lambda row: vote_taker(row, forest3_oob), axis=1)
game_table['forest3_oob_types'] = game_table.apply(lambda row: predictor_case(row, pred='forest3_oob', target='rating_over_70'), axis=1)
forest3_oob_types = game_table['forest3_oob_types'].value_counts()
print(forest3_oob_types.sum())  # length of testing table
print((accuracy(forest3_oob_types), f1(forest3_oob_types), informedness(forest3_oob_types)))

16719
(0.7525569711107124, 0.0038526366482061159, 0.0019300361881784411)


<div class=h1_cell>
<p>
All measures, accuracy, F1 score, and informedness, decreased significantly with more than 10 trees.
<p>
It seems like 10 is the sweet spot.
</div>

<h1>

Decision Trees

</h1>
<div class=h1_cell>
<p>
We are now going to explore the decision tree model. We are going to be testing how depth affects the accuracy of this model.

<p>
We're going to use a k-fold algoritm with random slices for our testing
</div>

In [55]:
def k_fold_random(table, k, target, hypers, candidate_columns):
    result_columns = ['name', 'true_positive', 'false_positive', 'true_negative', 'false_negative', 'accuracy', 'f1', 'informedness']
    k_fold_results_table = pd.DataFrame(columns=result_columns)
    
    total_len = len(table.index)
    split_size = int(total_len/(1.0*k))
    slices = []

    #generate the slices
    for i in range(k-1):
        a_slice =  table.sample(n=((i+1)*split_size - i*split_size))
        table = table.loc[~table.index.isin(a_slice.index)]
        slices.append( a_slice )
    slices.append( table[(k-1)*split_size:] )
    
    #generate test results
    for i in range(k-1):
        test_table = slices[i]
        train_table = compute_training(slices, i)
        fold_tree = build_tree_iter(train_table, candidate_columns, target, hypers)  # train
        fold_cases = caser(test_table, fold_tree, target)  # test

        k_fold_results_table = k_fold_results_table.append(fold_cases,ignore_index=True)
        end = k_fold_results_table.last_valid_index()
        k_fold_results_table.name.iloc[end] =  'fold '+str(i+1)+' test'
        k_fold_results_table.loc[end, 'accuracy'] =  accuracy(fold_cases)
        k_fold_results_table.f1.iloc[end] =  f1(fold_cases)
        k_fold_results_table.informedness.iloc[end] =  informedness(fold_cases)
        
    return k_fold_results_table

In [53]:
# going to use the same columns as with the random forests
kfold_columns = [
    #North American Sales
    'nas_low', 'nas_average', 'nas_high',
    #European Sales
    'eus_low', 'eus_average', 'eus_high',
    #Japanese Sales
    'jps_low', 'jps_average', 'jps_high',
    #Other Sales
    'ots_low', 'ots_average', 'ots_high',
    #Global Sales
    'gls_low', 'gls_average', 'gls_high'
]

In [56]:
#depth of 1
depth1 = k_fold_random(game_table, 10, 'rating_over_70', {'max-depth': 1}, kfold_columns)
depth1.describe()

,true_positive,false_positive,true_negative,false_negative,accuracy,f1,informedness
count,8.000000,5.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,3.125000,1.200000,1259.555556,408.000000,0.755436,0.013530,0.006307
std,1.246423,0.447214,23.585542,24.243556,0.014400,0.007603,0.004095
min,1.000000,1.000000,1227.000000,371.000000,0.736685,0.000000,-0.000810
25%,2.000000,1.000000,1248.000000,399.000000,0.747457,0.009479,0.003821
50%,4.000000,1.000000,1257.000000,410.000000,0.754638,0.017857,0.008215
75%,4.000000,1.000000,1268.000000,421.000000,0.761221,0.019370,0.009780
max,4.000000,2.000000,1296.000000,439.000000,0.776780,0.020942,0.009926


In [57]:
#try depth of 2
depth2 = k_fold_random(game_table, 10, 'rating_over_70', {'max-depth': 2}, kfold_columns)
depth2.describe()

,true_positive,false_positive,true_negative,false_negative,accuracy,f1,informedness
count,8.000000,3.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2.875000,1.666667,1257.111111,410.777778,0.753840,0.012178,0.005691
std,1.356203,1.154701,28.863664,27.779389,0.017045,0.007051,0.003832
min,2.000000,1.000000,1222.000000,358.000000,0.732496,0.000000,-0.000808
25%,2.000000,1.000000,1236.000000,397.000000,0.739677,0.009975,0.005013
50%,2.000000,1.000000,1262.000000,406.000000,0.757032,0.010000,0.005025
75%,3.500000,2.000000,1272.000000,434.000000,0.762418,0.014563,0.007335
max,5.000000,3.000000,1311.000000,444.000000,0.785757,0.023095,0.011682


<div class=h1_cell>
<p>
Mean accuracy, f1, and informedness seems to be lower. The max on the other hand is higher than depth 1.
</div>

In [61]:
#try depth of 4
depth4 = k_fold_random(game_table, 10, 'rating_over_70', {'max-depth': 4}, kfold_columns)
depth4.describe()

,true_positive,false_positive,true_negative,false_negative,accuracy,f1,informedness
count,9.000000,5.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,3.222222,1.200000,1256.222222,410.888889,0.753707,0.015481,0.007311
std,2.223611,0.447214,9.743944,11.061696,0.006584,0.010831,0.005492
min,1.000000,1.000000,1242.000000,394.000000,0.745661,0.004695,0.002353
25%,1.000000,1.000000,1247.000000,402.000000,0.746858,0.004866,0.002439
50%,3.000000,1.000000,1256.000000,410.000000,0.753441,0.014742,0.006656
75%,4.000000,1.000000,1264.000000,423.000000,0.758827,0.019002,0.008819
max,8.000000,2.000000,1269.000000,424.000000,0.764213,0.039024,0.019900


<div class=h1_cell>
<p>
Accuracy down but f1 and informedness are up.
Let's try one more.
</div>

In [62]:
#try depth of 6
depth6 = k_fold_random(game_table, 10, 'rating_over_70', {'max-depth': 6}, kfold_columns)
depth6.describe()

,true_positive,false_positive,true_negative,false_negative,accuracy,f1,informedness
count,9.000000,6.0,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2.888889,1.0,1257.333333,410.111111,0.754172,0.013819,0.006448
std,1.452966,0.0,22.152878,21.819970,0.013124,0.007015,0.003677
min,1.000000,1.0,1232.000000,387.000000,0.738480,0.005102,0.001798
25%,2.000000,1.0,1236.000000,389.000000,0.740874,0.009070,0.003755
50%,3.000000,1.0,1251.000000,415.000000,0.751047,0.015152,0.006871
75%,4.000000,1.0,1279.000000,432.000000,0.766607,0.018648,0.008748
max,5.000000,1.0,1282.000000,436.000000,0.768402,0.025189,0.012755


<h1>
Results
</h1>
<div class=h1_cell>
<p>
It seems like overall the random k-fold model performed better than the random forest model. 
<p>
I found that a 10 tree forest was ideal and provided us with scores of (0.753274, 0.0129217, 0.005957) for accuracy, f1, and informedness respectively. However, our 4 deep 10-fold model performed better with mean scores of (0.753707, 0.015481, 0.007311) for accuracy, f1, and informedness respectively.
<p>
The random forests behaved how I thought they would. Adding more trees increased the scores on accuracy, f1, and informedness compared to the orignal 5 trees. However, if you continue to add trees, you will eventually start seeing your scores decline with the more trees you add.
<p>
Our 10-fold model was interesting because at times increasing the depth resulted in increases to f1 and informedness but decreases to accuracy. To me it seems like accuracy is the most important trait so I would try to avoid decreases to it and find a good balance.
</div>

In [63]:
import os

home_path =  os.path.expanduser('~')

file_path = '/Documents/CIS 399/'

file_name = 'wrangled_game_table_trees.csv'

game_table.to_csv(home_path + file_path + file_name, index=False)